# Import libraries

In [1]:
import os
import k3d
import json
import numpy as np
from tqdm import tqdm
from easydict import EasyDict as edict

# Specifying trained model

In [2]:
EXP_NAME = '108364b8-aad7-4c4d-b093-4a69a49ececc'

with open(f'experiments/{EXP_NAME}/config.json') as json_file:
    config = json.load(json_file)

config = edict(config)

os.makedirs(f'experiments/{EXP_NAME}/visualizations', exist_ok=True)

# Visualizating

In [5]:
for subject in ['4']:#tqdm(config.subjects):
    
    with open(f'experiments/{EXP_NAME}/predictions/{subject}.json') as json_file:
        net_output = json.load(json_file)
        
    coordinates = np.array(net_output['coordinates'])
    predictions = np.array(net_output['predictions'])
    labels = np.array(net_output['labels'])
    air_maks = np.array(net_output['air_maks'])

    coordinates = coordinates*(1-air_maks[:, None]).astype(np.float32)
    predictions = predictions*(1-air_maks)
    predictions = (predictions > 0.8).astype(np.float32)
    labels = labels*(1-air_maks)
    tp = np.logical_and(labels, predictions)
    fp = np.logical_and(1-labels, predictions)
    fn = np.logical_and(labels, 1-predictions)
    
    pred_colors = np.array([0xCCCCCC for _ in range(len(predictions))])
    
#     pred_colors = [0x00FF00 if tp[i] else x for i, x in enumerate(pred_colors)]
#     pred_colors = [0xFF9900 if fp[i] else x for i, x in enumerate(pred_colors)]
#     pred_colors = [0xFF0000 if fn[i] else x for i, x in enumerate(pred_colors)]



    #opacities = [0.5 if (x == 0xCCCCCC) else 1 for x in pred_colors]
    #point_sizes = [1 if (x == 0x00FF00 or x == 0xFF0000) else 0.5 for x in pred_colors]
    opacities = [1 if (x == 0xCCCCCC) else 1 for x in pred_colors]
    point_sizes = [1 if (x == 0x00FF00 or x == 0xFF0000) else 1 for x in pred_colors]
    
    
    plot = k3d.plot(name='points')
    plt_points = k3d.points(positions=coordinates,
                            colors=pred_colors,
                            opacities=opacities,
                            point_sizes=point_sizes)
    plot += plt_points
    plt_points.shader='3d'

    with open(f'experiments/{EXP_NAME}/visualizations/{subject}.html', 'w') as f:
        f.write(plot.get_snapshot(9))